# Get-started (Azure version)(WIP)

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed PromptFlow SDK

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create and develop a new promptflow batch run
- Evaluate the batch run with a evaluation flow
- Deploy the flow to a remote endpoint.

**Motivations** - This guide will walk you through the main user journey of prompt flow code-first experience. You will learn how to create and develop your first prompt flow, test and evaluate it, then deploy it to production.

## Step 0: Install promptflow sdk
```bash
pip install promptflow-sdk[azure,builtins] --extra-index-url https://azuremlsdktestpypi.azureedge.net/promptflow/
```

## Step 1: Create necessary connections
Connection helps securely store and manage secret keys or other sensitive credentials required for interacting with LLM and other external tools for example Azure Content Safety.

In this notebook, we will use flow `web-classification` which uses connection `azure_open_ai_connection` inside, we need to set up the connection if we haven't added it before.

Please go to workspace portal, click `Prompt flow` -> `Connections` -> `Create`, then follow the instruction to create your own connections.

## Step 2: Create and develop a new batch run

`web-classification` is a flow demonstrating multi-class classification with LLM. Given an url, it will classify the url into one web category with just a few shots, simple summarization and classification prompts.

In [ ]:
import promptflow.azure as pf 
from promptflow.azure.entities import BulkFlowRunInput

# configure azureml workspace ml_client
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id="<subscription-id>",
    resource_group_name="<resource-group-name>",
    workspace_name="<workspace-name>",
)
client = pf.PFClient(ml_client=ml_client)


### Set flow path and bulk inputs

In [ ]:
# load flow
flow_dir = "../../flows/standard/web-classification"
data = "../../flows/standard/web-classification/data.jsonl"
runtime = "<runtime-name>"

flow = pf.load_flow(flow_dir)
print(flow)

### Submit bulk run

In [ ]:
# standard flow using remote data
run = flow.submit_bulk_run(
    data=data,
    connections="<connection-name>",  # To be confirmed.
    runtime=runtime,
)
print(run)

In [ ]:
pf.stream(run)

In [ ]:
pf.show_details(run)

In [ ]:
pf.show_metrics(run)

## Step 2: Evaluate your bulk flow run
Then you can use an evaluation method to evaluate your flow. The evaluation methods are also flows which use Python or LLM etc., to calculate metrics like accuracy, relevance score.

In this notebook, we use `classification-accuracy-eval` flow to evaluate. This is a flow illustrating how to evaluate the performance of a classification system. It involves comparing each prediction to the groundtruth and assigns a "Correct" or "Incorrect" grade, and aggregating the results to produce metrics such as accuracy, which reflects how good the system is at classifying the data.

In [ ]:
from promptflow.azure.entities import BulkFlowRunInput


classification_accuracy_eval = pf.load_flow("../../flows/standard/classification_accuracy_evaluation")

bulk_flow_run_input = BulkFlowRunInput(
    data=data,
    variants=[run],
    inputs_mapping={"groundtruth": "${data.answer}","prediction": "${variant.outputs.category}"},
)

eval_run = classification_accuracy_eval.submit_bulk_run(
    data=bulk_flow_run_input,
    runtime=runtime,
)

print(eval_run)

In [ ]:
pf.stream(eval_run)

In [ ]:
pf.show_details(eval_run)

In [ ]:
pf.show_metrics(eval_run)

In [ ]:
pf.visualize([run, eval_run])

## Step 3: Deploy the flow (WIP)